In [158]:
import json
import numpy as np
import pandas as pd

import MeCab

from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn.manifold import TSNE

import plotly.express as px
from datetime import datetime


In [159]:
stories_json = []
with open('./output_storiesv2.json', 'r') as f:
    stories_json = stories_json + json.loads(f.read())

print(len(stories_json))
print(stories_json[1].keys())

407
dict_keys(['index', 'title', 'broadcasting_date', 'abstracts_list'])


In [160]:
stories_df = pd.DataFrame(stories_json)
stories_df["abstract_joined"] = stories_df["abstracts_list"].str.join(" ")
stories_df.head()

,index,title,broadcasting_date,abstracts_list,abstract_joined
0,0325,春だ！映画だ！３時間アニメ祭り\r\n「映画ドラえもん のび太の新魔界大冒険～７人の魔法使い...,2013年3月15日,[ 何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいな...,何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいない...
1,0326,「ばくはつコショウ」「何が何でもお花見を」[2013年4月12日放送],2013年4月12日,[ ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。この...,ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。このコ...
2,0327,「やりクリしてハワイ旅行」「しずかちゃんのはごろも」[2013年4月26日放送],2013年4月26日,[ スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとする...,スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとするが...
3,0328,「ひるねは天国で」「どくさいスイッチ」[2013年5月3日放送],2013年5月3日,[ のび太とドラえもんが部屋で昼寝をしていると、突然大きな音が聞こえてくる。となりの家が改築...,のび太とドラえもんが部屋で昼寝をしていると、突然大きな音が聞こえてくる。となりの家が改築工...
4,0329,「インスタントママ」「狙われたジャイアン」[2013年5月10日放送],2013年5月10日,[ そうじにせんたく、町内会の会合と毎日大忙しのママが、「体がもうひとつほしい」とため息をつ...,そうじにせんたく、町内会の会合と毎日大忙しのママが、「体がもうひとつほしい」とため息をつい...


In [161]:
t = MeCab.Tagger('-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd')
def mecab_tokenizer(text):
    parsed_lines = t.parse(text).split("\n")[:-2]
    surfaces = [l.split('\t')[0] for l in parsed_lines]
    features = [l.split('\t')[1] for l in parsed_lines]
    # 原型を取得
    bases = [f.split(',')[6] for f in features]
    # 品詞を取得
    pos = [f.split(',')[0] for f in features]

    # 各単語を原型に変換する
    token_list = [b if b != '*' else s for s, b in zip(surfaces, bases)]
    # 品詞を絞り込み
    target_pos = ["名詞", "動詞", "形容詞"]
    token_list = [t for t, p in zip(token_list, pos) if p in target_pos]
    # アルファベットを小文字に統一
    token_list = [t.lower() for t in token_list]

    return [ text for text in filter(lambda x: not x.isascii(), token_list)]

In [162]:
# ストーリーが取得できたものに絞る
stories_df = stories_df[stories_df['abstract_joined'].str.len() > 0] 
sentences = stories_df["abstract_joined"].apply(mecab_tokenizer)
print(sentences)

0      [何, やる, うまい, いく, のび太, 魔法, 魔法, 使える, 便利, 楽しい, なる...
1      [朝, 寝坊, する, のび太, 泣きつく, れる, ドラえもん, 爆発, コショウ, 取り...
2      [スネ夫, ハワイ旅行, じまん, する, れる, のび太, 自分, ハワイ, 行く, ママ...
3      [のび太, ドラえもん, 部屋, 昼寝, する, いる, 音, 聞こえる, くる, となり,...
4      [そうじ, せんたく, 町内会, 会合, 毎日, 大忙し, ママ, 体, ひとつ, ほしい,...
                             ...                        
402    [しずか, 遊び, さそえる, ジャイアン, 意味, ない, ボコボコ, する, れる, し...
403    [スネ夫, 手, 入る, 恐竜, きょう, りゅう, プラモデル, じまん, する, れる,...
404    [秋, 楽しむ, ため, 別荘, べっそう, 行く, スネ夫, 季節感, ない, 顔, バカ...
405    [空き地, サッカー, する, 遊ぶ, いる, ジャイアン, スネ夫, たち, ジャイアン,...
406    [使い, の, ちゅ, 腕木, げん, 悪い, そう, ジャイアン, 見かける, 別, 道,...
Name: abstract_joined, Length: 407, dtype: object


In [163]:
abstract_list = sentences.tolist()
trainings = [TaggedDocument(words = data, tags = [i]) for i,data in enumerate(abstract_list)]
m = Doc2Vec(documents= trainings, dm = 1, window=8, min_count=10, workers=4)


In [164]:
vector_list = m.dv.vectors
tsne = TSNE(n_components=2)
vector_enbedded = tsne.fit_transform(vector_list)
print(vector_enbedded)

/Users/takamirei/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:795: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/Users/takamirei/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:805: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



[[-1.82622662e+01 -1.47641478e+01]
 [-8.24732494e+00 -3.24432158e+00]
 [-1.12485409e+01 -1.43961728e+00]
 [-1.89774723e+01 -1.54324932e+01]
 [-1.53277750e+01 -8.59120274e+00]
 [-1.95455971e+01 -1.27554598e+01]
 [-2.34214954e+01 -1.31790581e+01]
 [-2.09072227e+01 -1.74780426e+01]
 [-1.80605888e+01 -1.92772236e+01]
 [-1.38804255e+01 -1.19168434e+01]
 [-1.25768719e+01 -5.66787767e+00]
 [ 8.77319622e+00  7.72685349e-01]
 [ 1.12907991e+01 -2.26879907e+00]
 [-7.27656460e+00 -1.46055832e+01]
 [-4.03383923e+00 -3.72466636e+00]
 [-4.92450333e+00 -6.03634644e+00]
 [-7.02990246e+00 -4.27727079e+00]
 [-6.48918056e+00 -9.55318832e+00]
 [-5.79792976e+00  7.03214741e+00]
 [-1.13154764e+01 -3.50126719e+00]
 [-1.00712242e+01 -1.68103275e+01]
 [-7.49541903e+00 -5.37870169e+00]
 [-1.91750412e+01 -2.00221081e+01]
 [-1.48151469e+00 -6.68399334e+00]
 [-1.77133656e+01 -7.12459850e+00]
 [ 7.44431877e+00  2.62087536e+00]
 [ 9.68251514e+00  3.64797282e+00]
 [-2.94139004e+00  1.09543133e+01]
 [-4.17099190e+00 -6

In [165]:
vector_df = pd.DataFrame.from_dict({
    'vector': [v for v in vector_list],
    'vector_x': [v[0] for v in vector_enbedded],
    'vector_y': [v[1] for v in vector_enbedded]
})
stories_df_merged = stories_df.join(vector_df)
stories_df_merged['broadcasting_dt'] = stories_df_merged['broadcasting_date'].apply(lambda d: datetime.strptime(d, '%Y年%m月%d日'))
stories_df_merged['broadcasting_year'] = stories_df_merged['broadcasting_dt'].dt.year

In [166]:
fig = px.scatter(stories_df_merged, x='vector_x', y='vector_y', color='broadcasting_year', hover_data=["index", "title"])
fig.show()

In [167]:
df = px.data.tips()
fig = px.scatter(stories_df_merged, x='vector_x', y='vector_y', hover_data=["index", "title"], facet_row="broadcasting_year", width=800, height=2000)
fig.show()

In [168]:
sampled_df = stories_df_merged.sample(1)
print(sampled_df.head())
sampled_df = sampled_df.rename(columns={"vector": "sampled_vec"})[["sampled_vec"]]


    index                                   title broadcasting_date  \
275  0600  「気球で世界一周！？」「大ピンチ！スネ夫の答案」[2020年4月25日放送]        2020年4月25日   

                                        abstracts_list  \
275  [世界一周旅行に出発する気球を紹介（しょうかい）しているテレビ番組を見たのび太とドラえもん。...   

                                       abstract_joined  \
275  世界一周旅行に出発する気球を紹介（しょうかい）しているテレビ番組を見たのび太とドラえもん。次...   

                                                vector  vector_x   vector_y  \
275  [-0.043350436, 0.05344963, 0.008289449, -0.025...  5.032874  11.766717   

    broadcasting_dt  broadcasting_year  
275      2020-04-25               2020  


In [169]:
stories_df_merged = stories_df_merged.assign(key=1).merge(sampled_df.assign(key=1), on='key').drop('key', axis=1)
stories_df_merged

,index,title,broadcasting_date,abstracts_list,abstract_joined,vector,vector_x,vector_y,broadcasting_dt,broadcasting_year,sampled_vec
0,0325,春だ！映画だ！３時間アニメ祭り\r\n「映画ドラえもん のび太の新魔界大冒険～７人の魔法使い...,2013年3月15日,[ 何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいな...,何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいない...,"[-0.007591696, 0.011573967, -0.018565057, 0.01...",-18.262266,-14.764148,2013-03-15,2013,"[-0.043350436, 0.05344963, 0.008289449, -0.025..."
1,0326,「ばくはつコショウ」「何が何でもお花見を」[2013年4月12日放送],2013年4月12日,[ ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。この...,ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。このコ...,"[-0.02041695, 0.027010115, -0.018392317, 0.006...",-8.247325,-3.244322,2013-04-12,2013,"[-0.043350436, 0.05344963, 0.008289449, -0.025..."
2,0327,「やりクリしてハワイ旅行」「しずかちゃんのはごろも」[2013年4月26日放送],2013年4月26日,[ スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとする...,スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとするが...,"[-0.025363397, 0.041915685, 0.002021089, -0.01...",-11.248541,-1.439617,2013-04-26,2013,"[-0.043350436, 0.05344963, 0.008289449, -0.025..."
3,0328,「ひるねは天国で」「どくさいスイッチ」[2013年5月3日放送],2013年5月3日,[ のび太とドラえもんが部屋で昼寝をしていると、突然大きな音が聞こえてくる。となりの家が改築...,のび太とドラえもんが部屋で昼寝をしていると、突然大きな音が聞こえてくる。となりの家が改築工...,"[-0.0069613033, 0.02737202, -0.0029897853, 0.0...",-18.977472,-15.432493,2013-05-03,2013,"[-0.043350436, 0.05344963, 0.008289449, -0.025..."
4,0329,「インスタントママ」「狙われたジャイアン」[2013年5月10日放送],2013年5月10日,[ そうじにせんたく、町内会の会合と毎日大忙しのママが、「体がもうひとつほしい」とため息をつ...,そうじにせんたく、町内会の会合と毎日大忙しのママが、「体がもうひとつほしい」とため息をつい...,"[-0.02255871, 0.025910847, -0.018300397, -0.00...",-15.327775,-8.591203,2013-05-10,2013,"[-0.043350436, 0.05344963, 0.008289449, -0.025..."
...,...,...,...,...,...,...,...,...,...,...,...
402,0727,「オノマトペがいっぱい」「本はおいしくよもう」[2022年10月8日放送],2022年10月8日,[しずかを遊びにさそえず、ジャイアンからは意味もなくボコボコにされてしまったのび太が家に帰る...,しずかを遊びにさそえず、ジャイアンからは意味もなくボコボコにされてしまったのび太が家に帰ると...,"[0.005109428, -0.010532638, -0.012803972, 0.02...",-13.541646,-18.486856,2022-10-08,2022,"[-0.043350436, 0.05344963, 0.008289449, -0.025..."
403,0728,「まるでホンモノ！？自然観察プラモシリーズ」「のぞき穴ボード」[2022年10月15日放送],2022年10月15日,[スネ夫から、なかなか手に入らないという恐竜（きょうりゅう）のプラモデルをじまんされたのび太...,スネ夫から、なかなか手に入らないという恐竜（きょうりゅう）のプラモデルをじまんされたのび太は...,"[-0.003938366, 0.055168815, -0.010294088, -0.0...",-9.805190,6.339504,2022-10-15,2022,"[-0.043350436, 0.05344963, 0.008289449, -0.025..."
404,0729,「オールシーズンバッジ」「グンニャリジャイアン」[2022年10月22日放送],2022年10月22日,[秋を楽しむため、これから別荘（べっそう）に行くというスネ夫に、季節感のない顔だとバカにされ...,秋を楽しむため、これから別荘（べっそう）に行くというスネ夫に、季節感のない顔だとバカにされた...,"[-0.037785094, 0.016909577, 0.0160694, -0.0207...",-5.205653,-3.503592,2022-10-22,2022,"[-0.043350436, 0.05344963, 0.008289449, -0.025..."
405,0730,「なんでもクレーンゲーム」「むすびの糸」[2022年10月29日放送],2022年10月29日,[空き地でサッカーをして遊んでいたジャイアンとスネ夫たち。ところが、ジャイアンがけり上げたボ...,空き地でサッカーをして遊んでいたジャイアンとスネ夫たち。ところが、ジャイアンがけり上げたボー...,"[-0.030187866, 0.024459282, 0.038469743, -0.01...",-23.054029,-14.002565,2022-10-29,2022,"[-0.043350436, 0.05344963, 0.008289449, -0.025..."


In [170]:
stories_df_merged

,index,title,broadcasting_date,abstracts_list,abstract_joined,vector,vector_x,vector_y,broadcasting_dt,broadcasting_year,sampled_vec
0,0325,春だ！映画だ！３時間アニメ祭り\r\n「映画ドラえもん のび太の新魔界大冒険～７人の魔法使い...,2013年3月15日,[ 何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいな...,何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいない...,"[-0.007591696, 0.011573967, -0.018565057, 0.01...",-18.262266,-14.764148,2013-03-15,2013,"[-0.043350436, 0.05344963, 0.008289449, -0.025..."
1,0326,「ばくはつコショウ」「何が何でもお花見を」[2013年4月12日放送],2013年4月12日,[ ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。この...,ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。このコ...,"[-0.02041695, 0.027010115, -0.018392317, 0.006...",-8.247325,-3.244322,2013-04-12,2013,"[-0.043350436, 0.05344963, 0.008289449, -0.025..."
2,0327,「やりクリしてハワイ旅行」「しずかちゃんのはごろも」[2013年4月26日放送],2013年4月26日,[ スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとする...,スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとするが...,"[-0.025363397, 0.041915685, 0.002021089, -0.01...",-11.248541,-1.439617,2013-04-26,2013,"[-0.043350436, 0.05344963, 0.008289449, -0.025..."
3,0328,「ひるねは天国で」「どくさいスイッチ」[2013年5月3日放送],2013年5月3日,[ のび太とドラえもんが部屋で昼寝をしていると、突然大きな音が聞こえてくる。となりの家が改築...,のび太とドラえもんが部屋で昼寝をしていると、突然大きな音が聞こえてくる。となりの家が改築工...,"[-0.0069613033, 0.02737202, -0.0029897853, 0.0...",-18.977472,-15.432493,2013-05-03,2013,"[-0.043350436, 0.05344963, 0.008289449, -0.025..."
4,0329,「インスタントママ」「狙われたジャイアン」[2013年5月10日放送],2013年5月10日,[ そうじにせんたく、町内会の会合と毎日大忙しのママが、「体がもうひとつほしい」とため息をつ...,そうじにせんたく、町内会の会合と毎日大忙しのママが、「体がもうひとつほしい」とため息をつい...,"[-0.02255871, 0.025910847, -0.018300397, -0.00...",-15.327775,-8.591203,2013-05-10,2013,"[-0.043350436, 0.05344963, 0.008289449, -0.025..."
...,...,...,...,...,...,...,...,...,...,...,...
402,0727,「オノマトペがいっぱい」「本はおいしくよもう」[2022年10月8日放送],2022年10月8日,[しずかを遊びにさそえず、ジャイアンからは意味もなくボコボコにされてしまったのび太が家に帰る...,しずかを遊びにさそえず、ジャイアンからは意味もなくボコボコにされてしまったのび太が家に帰ると...,"[0.005109428, -0.010532638, -0.012803972, 0.02...",-13.541646,-18.486856,2022-10-08,2022,"[-0.043350436, 0.05344963, 0.008289449, -0.025..."
403,0728,「まるでホンモノ！？自然観察プラモシリーズ」「のぞき穴ボード」[2022年10月15日放送],2022年10月15日,[スネ夫から、なかなか手に入らないという恐竜（きょうりゅう）のプラモデルをじまんされたのび太...,スネ夫から、なかなか手に入らないという恐竜（きょうりゅう）のプラモデルをじまんされたのび太は...,"[-0.003938366, 0.055168815, -0.010294088, -0.0...",-9.805190,6.339504,2022-10-15,2022,"[-0.043350436, 0.05344963, 0.008289449, -0.025..."
404,0729,「オールシーズンバッジ」「グンニャリジャイアン」[2022年10月22日放送],2022年10月22日,[秋を楽しむため、これから別荘（べっそう）に行くというスネ夫に、季節感のない顔だとバカにされ...,秋を楽しむため、これから別荘（べっそう）に行くというスネ夫に、季節感のない顔だとバカにされた...,"[-0.037785094, 0.016909577, 0.0160694, -0.0207...",-5.205653,-3.503592,2022-10-22,2022,"[-0.043350436, 0.05344963, 0.008289449, -0.025..."
405,0730,「なんでもクレーンゲーム」「むすびの糸」[2022年10月29日放送],2022年10月29日,[空き地でサッカーをして遊んでいたジャイアンとスネ夫たち。ところが、ジャイアンがけり上げたボ...,空き地でサッカーをして遊んでいたジャイアンとスネ夫たち。ところが、ジャイアンがけり上げたボー...,"[-0.030187866, 0.024459282, 0.038469743, -0.01...",-23.054029,-14.002565,2022-10-29,2022,"[-0.043350436, 0.05344963, 0.008289449, -0.025..."


In [171]:
from scipy.spatial.distance import cosine

stories_df_merged["cosine_similarity"] =  stories_df_merged.apply(lambda v: cosine(v["vector"], v["sampled_vec"]), axis=1)
stories_df_merged_sort = stories_df_merged[["title", "abstract_joined", "cosine_similarity"]].sort_values("cosine_similarity")


In [172]:
stories_df_merged_sort

,title,abstract_joined,cosine_similarity
275,「気球で世界一周！？」「大ピンチ！スネ夫の答案」[2020年4月25日放送],世界一周旅行に出発する気球を紹介（しょうかい）しているテレビ番組を見たのび太とドラえもん。次...,0.000000
134,「ペラペラジャイアン飛んでった！」「みえないボディガード」[2016年10月21日放送],\r\n のび太たちが空き地で野球をしていると、１台の車がやって来て、運転手のおじさんが近く...,0.005049
191,ドラえもん クレヨンしんちゃん 春だ！映画だ！３時間アニメ祭り第２弾 「リモコンねこでとり返...,いとこに作ってもらったというリモコン飛行機をスネ夫にじまんされた帰り道、のび太はなんと二本足...,0.006011
37,「石器時代の王さまに」「鯉のぼりをつかまえろ！」[2014年4月25日放送],\r\n 図鑑（ずかん）を見ていたのび太は、便利（べんり）なモノを知らない石器時代に行って、...,0.006132
98,「のび泥棒をタイホせよ！」「あの子を笑わせろ！」[2015年11月27日放送],\r\n 小さな子どもたちがおにごっこをして遊んでいるのを見かけたのび太は、自分たちもやろう...,0.006294
...,...,...,...
155,GW特別企画！「火星ピクニック」「テレパしい」[2017年4月28日放送],火星人がいると信じているのび太は、「絶対にいない！」というスネ夫と言い合いになる。その様子を...,0.150741
363,「ジャイアンが警察官！？」「スネ夫の自慢はみんなの自慢」[2022年1月8日放送],警察官（けいさつかん）が主人公のドラマを見て、警察官にあこがれるのび太たち。将来（しょうらい...,0.152325
68,「無敵のスーパーランドセル」「もぐれ！ハマグリパック」[2015年2月27日放送],\r\n 学校からの帰り道、自転車をよけようとして電柱にぶつかったり、イヌにほえられてドブに...,0.180579
260,「なんでもエーアイアイ」「役立つもの販売機」[2020年1月11日放送],ドラ焼きを食べながら、足でマンガ本を取ろうとするのび太。しかも、あきれるドラえもんに、「ねて...,0.219004
